# Simulate the CT scan

In [ ]:
import os
import matplotlib.pyplot as plt
from JSON2gVXRDataReader import *

import SimpleITK as sitk

from cil.utilities.display import show_geometry
from cil.processors import TransmissionAbsorptionConverter
from cil.recon import FDK
from cil.plugins.astra import FBP
from cil.utilities.jupyter import islicer, link_islicer

In [ ]:
current_folder = str(globals()['_dh'][0])
data_path = "25OctoberData/"
# data_path = "24NovemberData/"
#data_path="15NovemberData/"
#data_path="4thOctober_image/"

simulation_JSON_fname = current_folder + "/" + data_path + "/simulation.json"
visible_light_JSON_fname = current_folder + "/" + data_path + "/visible_light.json"

In [ ]:
reader = JSON2gVXRDataReader(file_name=simulation_JSON_fname)
simulation_data = reader.read()
simulation_data = TransmissionAbsorptionConverter(white_level=simulation_data.max(), min_intensity=1e-5)(simulation_data)

reader = JSON2gVXRDataReader(file_name=visible_light_JSON_fname)
visible_light_data = reader.read()
visible_light_data = TransmissionAbsorptionConverter(white_level=visible_light_data.max(), min_intensity=1e-5)(visible_light_data)

In [ ]:
def reconstruct(data, backend):

    data.reorder(order=backend)
    ig = data.geometry.get_ImageGeometry()

    if backend == "tigre":
        recons = FDK(data, ig).run()
    elif backend == "astra":
        fbp =  FBP(ig, simulation_data.geometry)
        recons = fbp(data)        

    return ig, recons

def save(fname, recons):
# detector_size = np.array(gvxr.getDetectorSize("mm"))
# number_of_pixels = np.array(gvxr.getDetectorNumberOfPixels())
# spacing = detector_size / number_of_pixels

# print("CT volume saved in", "recons.mha")

    reconstruction_as_array = recons.as_array()
    sitk_image = sitk.GetImageFromArray(reconstruction_as_array)
    # sitk_image.SetSpacing([spacing[0], spacing[0], spacing[1]])
    sitk.WriteImage(sitk_image, fname, useCompression=True)


In [ ]:
simulation_ig, simulation_recons_tigre = reconstruct(simulation_data, "tigre")

In [ ]:
visible_light_ig, visible_light_recons_tigre = reconstruct(visible_light_data, "tigre")

In [ ]:
simulation_ig, simulation_recons_astra = reconstruct(simulation_data, "astra")

In [ ]:
visible_light_ig, visible_light_recons_astra = reconstruct(visible_light_data, "astra")

In [ ]:
print(simulation_data.geometry)

In [ ]:
show_geometry(simulation_data.geometry).save(current_folder + "/" + data_path + "/geometry.png")

In [ ]:
sl1_tigre = islicer(simulation_recons_tigre, direction='vertical', title="Simulated data reconstructed using Tigre")
sl2_tigre = islicer(visible_light_recons_tigre, direction='vertical', title="Experimental data reconstructed using Tigre")
link_islicer(sl1_tigre, sl2_tigre)

In [ ]:
sl3_tigre = islicer(simulation_recons_tigre, direction='horizontal_x', title="Simulated data reconstructed using Tigre")
sl4_tigre = islicer(visible_light_recons_tigre, direction='horizontal_x', title="Experimental data reconstructed using Tigre")
link_islicer(sl3_tigre, sl4_tigre)

In [ ]:
sl1_astra = islicer(simulation_recons_astra, direction='vertical', title="Simulated data reconstructed using Astra")
sl2_astra = islicer(visible_light_recons_astra, direction='vertical', title="Experimental data reconstructed using Astra")
link_islicer(sl1_astra, sl2_astra)

In [ ]:
sl3_astra = islicer(simulation_recons_astra, direction='horizontal_x', title="Simulated data reconstructed using Astra")
sl4_astra = islicer(visible_light_recons_astra, direction='horizontal_x', title="Experimental data reconstructed using Astra")
link_islicer(sl3_astra, sl4_astra)

In [ ]:
fname = current_folder + "/" + data_path + "/simulation-FDK-tigre.mha"
if not os.path.exists(fname):
    save(fname, simulation_recons_tigre)

In [ ]:
fname = current_folder + "/" + data_path + "/visible_light-FDK-tigre.mha"
if not os.path.exists(fname):
    save(fname, visible_light_recons_tigre)

In [ ]:
fname = current_folder + "/" + data_path + "/simulation-FDK-astra.mha"
if not os.path.exists(fname):
    save(fname, simulation_recons_astra)

In [ ]:
fname = current_folder + "/" + data_path + "/visible_light-FDK-astra.mha"
if not os.path.exists(fname):
    save(fname, visible_light_recons_astra)